In [62]:
import pandas as pd
import os
import glob
from dbconfig import config
import psycopg2
import re

In [41]:
dataset_dir = 'data'

In [42]:
csv_files = glob.glob('../data/*.CSV')

In [65]:
df_cursos = []
df_ies = []

for file in csv_files:
    try:
        if "CURSOS" in file:
            df_cursos.append(pd.read_csv(file, sep=','))
        else:
            df_ies.append(pd.read_csv(file, sep=','))
    except UnicodeDecodeError:
        if "CURSOS" in file:
            df_cursos.append(pd.read_csv(file, sep=',', encoding='ISO-8859-1'))
        else:
            df_ies.append(pd.read_csv(file, sep=',', encoding='ISO-8859-1'))

cursos = pd.concat(df_cursos)
ies = pd.concat(df_ies)

cursos.to_csv("../data/cursos.csv", header=cursos.columns, index=False, encoding='utf-8')
ies.to_csv("../data/instituicoes.csv", header=ies.columns, index=False, encoding='utf-8')

/tmp/ipykernel_25696/3309157878.py:7: DtypeWarning: Columns (1,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cursos.append(pd.read_csv(file, sep=','))
/tmp/ipykernel_25696/3309157878.py:7: DtypeWarning: Columns (1,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cursos.append(pd.read_csv(file, sep=','))
/home/andreb/Documents/censo-project/venv/lib64/python3.11/site-packages/pandas/core/internals/blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [ ]:
def transform(match):
    if match.group(1) == 'CURSOS':
        return 'cursos'
    elif match.group(1) == 'IES':
        return 'instituicoes'

for k in csv_files:
    if "CURSOS" in k:
        df = df_cursos[k] 
    else:
        df = df_ies[k] 

    tbl_name = re.sub(r'\.\./data/MICRODADOS_CADASTRO_(CURSOS|IES)_\d{4}\.CSV', 
                    transform, 
                    k)
    print(tbl_name)

    replacements = {
        'object': 'varchar',
        'float64': 'float',
        'int64': 'int'
    }

    #schema
    col_str = ",  ".join("{} {}".format(n, d) 
        for (n,d) in zip (df.columns, df.dtypes.replace(replacements)))
    
    conn = psycopg2.connect(**config())
    cursor = conn.cursor()

    #create table
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))

    df.to_csv(k, header=df.columns, index=False, encoding='utf-8')
    my_file = open(k)


    COPY_STATEMENT = """
        COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """
    cursor.copy_expert(sql=COPY_STATEMENT % tbl_name, file=my_file)
    conn.commit()
    cursor.close()

In [ ]:
replacements = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int'
}

#schema
col_str = ",  ".join("{} {}".format(n, d) 
    for (n,d) in zip (df.columns, df.dtypes.replace(replacements)))

conn = psycopg2.connect(**config())
cursor = conn.cursor()

#drop table
#create table
cursor.execute("create table %s (%s);" % (tbl_name, col_str))

df.to_csv(k, header=df.columns, index=False, encoding='utf-8')
my_file = open(k)


COPY_STATEMENT = """
    COPY %s FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
"""
cursor.copy_expert(sql=COPY_STATEMENT % tbl_name, file=my_file)
conn.commit()
cursor.close()

In [75]:
def load_data(file_path: str):
    tbl_name = ""
    if "cursos" in file_path:
        tbl_name = "cursos"
    else:
        tbl_name = "instituicoes"
    my_file = open(f"../data/{tbl_name}.csv")

    col_str = ",  ".join("{} {}".format(n, d) 
        for (n,d) in zip (df.columns, df.dtypes.replace(replacements)))
    
    conn = psycopg2.connect(**config())
    cursor = conn.cursor()
    
    cursor.execute("drop table if exists %s" % (tbl_name))

    cursor.execute("create table %s (%s);" % (tbl_name, col_str))

    COPY_STATEMENT = """
    COPY %s FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','
    """
    cursor.copy_expert(sql=COPY_STATEMENT % tbl_name, file=my_file)
    conn.commit()
    cursor.close()
    my_file.close()

In [76]:
load_data("../data/cursos.csv")